In [1]:
import numpy as np
import tensorflow as tf
import os

In [2]:
import matplotlib.pyplot as plt
import _pickle
import time

In [3]:
# todo
# compare likelihoods
# test with less popular features
# use different priors
# multiclass logistic regression significance tests
# fit mixture of gaussian vs gaussian
# check if weight activity killed
# tansfer learning for new class and robustness

In [4]:
file = open('mean_diff_saliencies_vanilla_temp1.pkl', "rb")
# file = open('mean_diff_saliency_mask_vanilla_temp1.pkl', "rb")
true_saliencies = _pickle.load(file) 
file = open('mean.pkl', "rb")
gaussian_means = _pickle.load(file) 
file = open('covariance.pkl', "rb")
gaussian_covariances = _pickle.load(file)

with open('turker_prob.pkl', "rb") as handle:
    turkers = _pickle.load(handle) 

with open('test_set_17_227.pkl', "rb") as handle:
    test_set = _pickle.load(handle) 
    
with open('turker_count.pkl', "rb") as handle:
    turker_count= _pickle.load(handle) 

In [5]:
true_saliencies['Alec Baldwin'].shape

(15, 227, 227, 3)

In [6]:
male_actors = ['Daniel Radcliffe', 'Gerard Butler', 'Michael Vartan', 'Alec Baldwin', 'Steve Carell',\
                    'Bill Hader', 'Fran Drescher', 'Matt Damon', 'Nicolas Cage']
female_actors = ['Cheryl Hines', 'Selena Gomez', 'Angie Harmon', 'Lorraine Bracco', 'Kristin Chenoweth',\
                      'America Ferrera', 'Anne Hathaway', 'Jennifer Aniston']
gender ={}
for actor in male_actors:
    gender[actor] = 'male'
for actor in female_actors:
    gender[actor] = 'female' 

In [7]:
# creating turker coiunt per actor
features = ['beard', 'eye brows', 'eyes', 'hairline', 'lips', 'moustache', 'nose']
actor_stats = {}
for actor in test_set:
    actor_stats[actor] = {}
    for feature in features:
        actor_stats[actor][feature] = 0  
    for stats in turker_count[actor]:
        for feature in features:
            try:
                 actor_stats[actor][feature] += stats[feature]
            except:
                continue

actor_stats_normalized = {}
for actor in actor_stats:
    actor_stats_normalized[actor] = {}
    total_count = sum(actor_stats[actor].values())
    for feature in features:
        actor_stats_normalized[actor][feature] = actor_stats[actor][feature]/total_count

In [8]:
def create_individual_prior(test_subject):

    if gender[test_subject] == 'male':
        actor_set = male_actors.copy()
    else:
        actor_set = female_actors.copy()
    
    actor_set.remove(test_subject)
    prior = {}
    for feature in features:
        feature_stats = [actor_stats_normalized[actor][feature] for actor in actor_set]
        feature_prior = sum(feature_stats)/len(feature_stats)
        prior[feature] = feature_prior
    
    return prior

In [9]:
# monte carlo rectangles generation
def generate_rectangles(means, covariances):
    mc_samples = {}
    for feature in features:
#         print(feature)
        rectangles = []
        while len(rectangles) < 100:
#             print(len(rectangles))
            width, height, y_centre, x_centre = np.random.multivariate_normal(means[feature], \
                                                                              covariances[feature])
            x_0= int(np.round(x_centre - width/2)) 
            x_1= int(np.round(x_centre + width/2)) 
            y_0= int(np.round(226 - y_centre - height/2)) 
            y_1= int(np.round(226 - y_centre + height/2)) 
            
#             x_0= int(np.round(x_centre - height/2)) 
#             x_1= int(np.round(x_centre + height/2)) 
#             y_0= int(np.round(y_centre - width/2)) 
#             y_1= int(np.round(y_centre + width/2)) 
            
#             print(np.array([x_centre, y_centre, height, width]))
#             print(np.array([x_0,x_1,y_0,y_1]))
            if 0<= x_0 < x_1 <=226 and 0<= y_0 < y_1 <=226:
                rectangles.append(np.array([x_0,x_1,y_0,y_1]))
                
        mc_samples[feature] = np.stack(rectangles)
        
    return mc_samples 

mc_rectangle_samples = generate_rectangles(gaussian_means, gaussian_covariances)

In [10]:
def likelihood(rectangles, saliency_map):
    scores = {}
    for feature in features:
        intensity_ratio_sum = 0
        for rec in rectangles[feature]:
            highlight = saliency_map[rec[2]:rec[3], rec[0]:rec[1],:]
            area = (rec[3]-rec[2])*(rec[1]-rec[0])
            intensity_ratio = (np.sum(highlight)/area)/np.sum(saliency_map)/(227**2)
            intensity_ratio_sum += intensity_ratio
        scores[feature] = intensity_ratio_sum/len(rectangles[feature])
    
    sum_scores = np.sum(np.array(list(scores.values())))
    probs = {}
    
    for feature in features:
        probs[feature] = scores[feature]/sum_scores
    
    return probs

In [11]:
def posterior(rectangles, saliency_map, test_subject):
    scores = {}
    prior = create_individual_prior(test_subject)
    for feature in features:
        intensity_ratio_sum = 0
        for rec in rectangles[feature]:
            highlight = saliency_map[rec[2]:rec[3], rec[0]:rec[1],:]
            area = (rec[3]-rec[2])*(rec[1]-rec[0])
            intensity_ratio = (np.sum(highlight)/area)/np.sum(saliency_map)/(227**2)
            intensity_ratio_sum += intensity_ratio
        scores[feature] = prior[feature]*intensity_ratio_sum/len(rectangles[feature])
    
    sum_scores = np.sum(np.array(list(scores.values())))
    probs = {}
    
    for feature in sorted(scores.keys()):
        probs[feature] = scores[feature]/sum_scores
    
    return probs

In [12]:
actor_true_saliencies = {}

for actor in true_saliencies:
    actor_saliency = np.mean(true_saliencies[actor], axis=0)
    actor_true_saliencies[actor] = actor_saliency

In [13]:
predictions = {}
for actor in sorted(test_set.keys()):
    saliency_map = actor_true_saliencies[actor]
    probs = likelihood(mc_rectangle_samples, saliency_map)
    predictions[actor]=probs

In [14]:
posterior_predictions = {}
for actor in sorted(test_set.keys()):
    saliency_map = actor_true_saliencies[actor]
    probs = posterior(mc_rectangle_samples, saliency_map, actor)
    posterior_predictions[actor]=probs

In [15]:
# main problems are with turker data
# maybe imbalanced, also tend to label similar feature
# maybe should try new set of actors who have very distinct features to do anchoring
# people choocse what seems to get highlighted in that image, but not what is salient about the person
# probaly better to frame questionas as what is salient about this individual

In [16]:
import operator

actors = sorted(test_set.keys())

f, axes = plt.subplots(17, 4, figsize=(60,170))

for i in range(len(actors)):
    actor = actors[i]
    ax = axes[i]

    ax[0].set_title(actor)
    ax[0].axis('off')
    ax[0].imshow(actor_true_saliencies[actor])
        
    prediction = posterior_predictions[actor]
    turker_data = actor_stats_normalized[actor]
    sorted_turker_data = sorted(turker_data.items(), key=operator.itemgetter(1))
    sorted_turker_data.reverse()
    
    X = np.arange(len(turker_data))
    ax[1].set_ylim([0, 0.7])
    ax[1].bar(X, [prediction[key[0]] for key in sorted_turker_data],\
           width=0.2, color='b', align='center', label='Posterior Prediction')
    ax[1].bar(X-0.2, [turker_data[key[0]] for key in sorted_turker_data],\
           width=0.2, color='g', align='center', label='Turkers')
    ax[1].legend()
    ax[1].set_xticks(X)
    ax[1].set_xticklabels([key[0] for key in sorted_turker_data])
    ax[1].set_title('turkers vs posterior', fontsize=17)
    
    
    prediction = create_individual_prior(actor)
    X = np.arange(len(turker_data))
    ax[2].set_ylim([0, 0.7])
    ax[2].bar(X, [prediction[key[0]] for key in sorted_turker_data],\
           width=0.2, color='b', align='center', label='Prior Prediction')
    ax[2].bar(X-0.2, [turker_data[key[0]] for key in sorted_turker_data],\
           width=0.2, color='g', align='center', label='Turkers')
    ax[2].legend()
    ax[2].set_xticks(X)
    ax[2].set_xticklabels([key[0] for key in sorted_turker_data])
    ax[2].set_title('turkers vs prior', fontsize=17)
    
    prediction = predictions[actor]
    X = np.arange(len(turker_data))
    ax[3].set_ylim([0, 0.7])
    ax[3].bar(X, [prediction[key[0]] for key in sorted_turker_data],\
           width=0.2, color='b', align='center', label='Backprop Prediction')
    ax[3].bar(X-0.2, [turker_data[key[0]] for key in sorted_turker_data],\
           width=0.2, color='g', align='center', label='Turkers')
    ax[3].legend()
    ax[3].set_xticks(X)
    ax[3].set_xticklabels([key[0] for key in sorted_turker_data])
    ax[3].set_title('turkers vs backprop', fontsize=17)

    print(actor)
    
    
plt.savefig('nomalized_prior_predictions_size_normalized_meandiff.png')
plt.close()

Alec Baldwin
America Ferrera
Angie Harmon
Anne Hathaway
Bill Hader
Cheryl Hines
Daniel Radcliffe
Fran Drescher
Gerard Butler
Jennifer Aniston
Kristin Chenoweth
Lorraine Bracco
Matt Damon
Michael Vartan
Nicolas Cage
Selena Gomez
Steve Carell


In [17]:
from scipy.stats import spearmanr
cors = []
pvals = []
for actor in sorted(test_set.keys()):
    print(actor)
    turker_data = actor_stats[actor]
    sorted_turker_data = sorted(turker_data.items(), key=operator.itemgetter(1))
    prediction = posterior_predictions[actor]
    predicted = np.array([prediction[key[0]] for key in sorted_turker_data])
    turker = np.array([turker_data[key[0]] for key in sorted_turker_data])
    cor, pval = spearmanr(predicted, turker) 
    cors.append(cor)
    pvals.append(pval)
    print(spearmanr(predicted, turker))
print('correlation')
print(np.mean(np.array(cors)))    
print('pvalue')
print(np.mean(np.array(pvals)))

Alec Baldwin
SpearmanrResult(correlation=0.95499371045729253, pvalue=0.00080553533630175773)
America Ferrera
SpearmanrResult(correlation=0.85243671416026634, pvalue=0.014814085490537589)
Angie Harmon
SpearmanrResult(correlation=0.99103120896511487, pvalue=1.4561252916129422e-05)
Anne Hathaway
SpearmanrResult(correlation=0.84688121493382551, pvalue=0.016197127467871632)
Bill Hader
SpearmanrResult(correlation=0.95499371045729253, pvalue=0.00080553533630175773)
Cheryl Hines
SpearmanrResult(correlation=0.84688121493382551, pvalue=0.016197127467871632)
Daniel Radcliffe
SpearmanrResult(correlation=0.8928571428571429, pvalue=0.0068071874089353918)
Fran Drescher
SpearmanrResult(correlation=0.92656164582637657, pvalue=0.0026974760978696771)
Gerard Butler
SpearmanrResult(correlation=0.60714285714285721, pvalue=0.14823116148116139)
Jennifer Aniston
SpearmanrResult(correlation=0.95499371045729253, pvalue=0.00080553533630175773)
Kristin Chenoweth
SpearmanrResult(correlation=0.95499371045729253, pva

In [18]:
from scipy.stats import spearmanr
cors = []
pvals = []
for actor in sorted(test_set.keys()):
    print(actor)
    turker_data = actor_stats[actor]
    sorted_turker_data = sorted(turker_data.items(), key=operator.itemgetter(1))
    prediction = create_individual_prior(actor)
    predicted = np.array([prediction[key[0]] for key in sorted_turker_data])
    turker = np.array([turker_data[key[0]] for key in sorted_turker_data])
    cor, pval = spearmanr(predicted, turker) 
    cors.append(cor)
    pvals.append(pval)
    print(spearmanr(predicted, turker))
print('correlation')
print(np.mean(np.array(cors)))    
print('pvalue')
print(np.mean(np.array(pvals)))

Alec Baldwin
SpearmanrResult(correlation=0.99103120896511487, pvalue=1.4561252916129422e-05)
America Ferrera
SpearmanrResult(correlation=0.96362411165943163, pvalue=0.00047530433988404632)
Angie Harmon
SpearmanrResult(correlation=0.99103120896511487, pvalue=1.4561252916129422e-05)
Anne Hathaway
SpearmanrResult(correlation=0.9189562119494703, pvalue=0.0034366261562175319)
Bill Hader
SpearmanrResult(correlation=0.9189562119494703, pvalue=0.0034366261562175319)
Cheryl Hines
SpearmanrResult(correlation=0.9189562119494703, pvalue=0.0034366261562175319)
Daniel Radcliffe
SpearmanrResult(correlation=0.8928571428571429, pvalue=0.0068071874089353918)
Fran Drescher
SpearmanrResult(correlation=0.88949917999332151, pvalue=0.0073394255616849513)
Gerard Butler
SpearmanrResult(correlation=0.75000000000000022, pvalue=0.052181400457057762)
Jennifer Aniston
SpearmanrResult(correlation=0.95499371045729253, pvalue=0.00080553533630175773)
Kristin Chenoweth
SpearmanrResult(correlation=0.99103120896511487, pv

In [19]:
from scipy.stats import spearmanr
cors = []
pvals = []
for actor in sorted(test_set.keys()):
    print(actor)
    turker_data = actor_stats[actor]
    sorted_turker_data = sorted(turker_data.items(), key=operator.itemgetter(1))
    prediction = predictions[actor]
    predicted = np.array([prediction[key[0]] for key in sorted_turker_data])
    turker = np.array([turker_data[key[0]] for key in sorted_turker_data])
    cor, pval = spearmanr(predicted, turker) 
    cors.append(cor)
    pvals.append(pval)
    print(spearmanr(predicted, turker))
print('correlation')
print(np.mean(np.array(cors)))    
print('pvalue')
print(np.mean(np.array(pvals)))

Alec Baldwin
SpearmanrResult(correlation=0.19820624179302301, pvalue=0.67008503391682661)
America Ferrera
SpearmanrResult(correlation=0.48181205582971581, pvalue=0.27358684135530992)
Angie Harmon
SpearmanrResult(correlation=0.70273122090253604, pvalue=0.078237453832071535)
Anne Hathaway
SpearmanrResult(correlation=-0.018018749253911181, pvalue=0.96941538680319872)
Bill Hader
SpearmanrResult(correlation=0.81084371642600317, pvalue=0.026916328843161991)
Cheryl Hines
SpearmanrResult(correlation=0.5, pvalue=0.25316999510032273)
Daniel Radcliffe
SpearmanrResult(correlation=0.8928571428571429, pvalue=0.0068071874089353918)
Fran Drescher
SpearmanrResult(correlation=0.33356219249749558, pvalue=0.46469707719905995)
Gerard Butler
SpearmanrResult(correlation=0.39285714285714296, pvalue=0.38331687042697266)
Jennifer Aniston
SpearmanrResult(correlation=0.0, pvalue=1.0)
Kristin Chenoweth
SpearmanrResult(correlation=-0.072074997015644723, pvalue=0.87795925082251292)
Lorraine Bracco
SpearmanrResult(co